In [ ]:
!pip install tensorflow keras matplotlib tqdm scikit-learn

### Numpy vs Tensorflow 

In [ ]:
import numpy as np

A = np.random.random((5000, 5000))
B = np.random.random((5000, 5000))

In [ ]:
%%time
result_np = A @ B

In [ ]:
import tensorflow as tf
s = tf.InteractiveSession()

# задаем входы для матриц
A_input = tf.placeholder('float32', shape=(5000, 5000), name='A')
B_input = tf.placeholder('float32', shape=(5000, 5000), name='B')

output = A_input @ B_input

In [ ]:
%%time
result_tf = output.eval({A_input: A,
                         B_input: B}, session=s)

In [ ]:
np.allclose(result_np, result_tf)

In [ ]:
v1 = # вектор
v2 = # вектор

mse = # mse 

compute_mse = lambda vector1, vector2: # возвращает число

In [ ]:
from sklearn.metrics import mean_squared_error

for n in [1,5,10,10**3]:
    
    elems = [np.arange(n),np.arange(n,0,-1), np.zeros(n),
             np.ones(n),np.random.random(n),np.random.randint(100,size=n)]
    
    for el in elems:
        for el_2 in elems:
            true_mse = np.array(mean_squared_error(el,el_2))
            my_mse = compute_mse(el, el_2)
            if not np.allclose(true_mse,my_mse):
                print(n, "Что-то не так", true_mse, my_mse)

## Переменные

In [ ]:
# заводим переменную
shared_vector_1 = tf.Variable(initial_value=np.ones(5))

In [ ]:
#инициализируем в графе
s.run(tf.global_variables_initializer())

print("initial value",s.run(shared_vector_1))

In [ ]:
#меняем значение
s.run(shared_vector_1.assign(np.arange(5)))
print ("new value", s.run(shared_vector_1))

# tf.gradients

In [ ]:
my_scalar = tf.placeholder('float32')

scalar_squared = my_scalar ** 2

derivative = tf.gradients(scalar_squared, my_scalar)[0]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

x = np.linspace(-3,3)
x_squared, x_squared_der = s.run([scalar_squared,derivative],
                                 {my_scalar:x})

plt.plot(x, x_squared,label="x^2")
plt.plot(x, x_squared_der, label="derivative")
plt.legend()

In [ ]:
my_vector = tf.placeholder('float32',[None])

weird_psychotic_function = tf.reduce_mean((my_vector + my_scalar) ** (1 + tf.nn.moments(my_vector, [0])[1]) + 1./ tf.atan(my_scalar))/(my_scalar ** 2 + 1) + 0.01 * tf.sin(2 * my_scalar ** 1.5) * (tf.reduce_sum(my_vector) * my_scalar ** 2) * tf.exp((my_scalar-4) ** 2) / (1 + tf.exp((my_scalar - 4) ** 2)) * (1. - (tf.exp(-(my_scalar - 4) ** 2)) / (1 + tf.exp(-(my_scalar - 4) ** 2))) ** 2


der_by_scalar, der_by_vector = # градиенты по переменным

In [ ]:
#Plotting your derivative
scalar_space = np.linspace(1,7,100)

y = [s.run(weird_psychotic_function,{my_scalar:x,my_vector:[1,2,3]})
     for x in scalar_space]

plt.plot(scalar_space,y,label='function')

y_der_by_scalar = [s.run(der_by_scalar,{my_scalar:x,my_vector:[1,2,3]})
     for x in scalar_space]

plt.plot(scalar_space,y_der_by_scalar,label='derivative')
plt.grid();plt.legend()


### Оптимизаторы


In [ ]:
y_guess = tf.Variable(np.zeros(2,dtype='float32'))
y_true = tf.range(1,3,dtype='float32')

loss = tf.reduce_mean((y_guess - y_true + tf.random_normal([2]))**2) 

optimizer = tf.train.MomentumOptimizer(0.01, 0.9).minimize(loss, var_list=y_guess)

In [ ]:
from IPython.display import clear_output

s.run(tf.global_variables_initializer())

guesses = [s.run(y_guess)]

for _ in range(100):
    s.run(optimizer)
    guesses.append(s.run(y_guess))
    
    clear_output(True)
    plt.plot(*zip(*guesses),marker='.')
    plt.scatter(*s.run(y_true),c='red')
    plt.show()

## Логистическая регрессия

In [ ]:
from sklearn.datasets import load_digits
mnist = load_digits(2)

X, y = mnist.data, mnist.target

X[0], y[0]

In [ ]:
weights = # веса
input_X = # картинки 
input_y = # ответы

In [ ]:
predicted_y = # предсказанные вероятности 
loss = # функция потерь

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(loss, var_list=weights)

In [ ]:
train_function = lambda X_b, y_b: # функция обучения, возвращает loss
predict_function = lambda X_b: # функция тестирования, возвращает предсказанные вероятности

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
s.run(tf.global_variables_initializer())

In [ ]:
from sklearn.metrics import roc_auc_score

for i in range(100):
    
    loss_i = train_function(X_train, y_train)
    
    print ("loss at iter %i:%.4f"%(i,loss_i))
    
    print ("train auc:", roc_auc_score(y_train, predict_function(X_train)))
    print ("test auc:", roc_auc_score(y_test, predict_function(X_test)))

    
print ("resulting weights:")
plt.imshow(weights.eval().reshape(8,-1))
plt.colorbar()

## Нейронная сеть

In [ ]:
class DenseNN:
    def __init__(self):
        self.training  = tf.placeholder_with_default(True, shape=())
        self.dense_1 = # полносвязный слой
        self.dense_2 = # последний полносвязный слой 
        
    def __call__(self, X):
        return self.dense_2(tf.layers.Dropout()(self.dense_1(X), training=self.training))

In [ ]:
nn = DenseNN()

In [ ]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

# np.array
train_data = mnist.train.images 
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images 
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

In [ ]:
plt.imshow(train_data[-1].reshape((28, 28)), cmap='gray')

In [ ]:
input_X = tf.placeholder('float32', shape=(None, ) + train_data.shape[1:])
input_y = tf.placeholder('int32', shape=(None, ))

nn = DenseNN()

# хотим получать не вероятности, а значения до softmax
logits = nn(input_X)

loss = tf.losses.sparse_softmax_cross_entropy(input_y, logits)

predictions = {"classes": tf.argmax(input=logits, axis=1),
               "probabilities": tf.nn.softmax(logits, name="softmax_tensor")}

In [ ]:
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss=loss,
                              global_step=tf.train.get_global_step())

In [ ]:
s.run(tf.global_variables_initializer())

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
batch_size = 1024
idx = np.arange(len(train_data))
for epoch in range(10):
    np.random.shuffle(idx)
    train_loss = 0
    cnt = 0
    for i in tqdm_notebook(range(0, len(train_data), batch_size)):
        cur_idx = idx[i: i + batch_size]
        cur_loss = s.run([loss, train_op], {input_X: train_data[idx], 
                                            input_y: train_labels[idx],
                                            nn.training: True})[0]
        train_loss += cur_loss
        cnt += 1
        
        if i % 64 == 63:
            print(train_loss / cnt)
        
    
        
    eval_loss = s.run(loss, {input_X: eval_data, 
                             input_y: eval_labels,
                             nn.training: False})
    classes = s.run(predictions['classes'], {input_X: eval_data, 
                                             input_y: eval_labels,
                                             nn.training: False})
    print(np.mean(classes == eval_labels))
    
    print(eval_loss)

# Keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(input_shape=train_data.shape[1:], units=10, activation='relu'))
model.add(Dense(input_shape=train_data.shape[1:], units=10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels, batch_size=64, epochs=10, validation_data=(eval_data, eval_labels))

# Zoo 

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
resnet = ResNet50()

In [ ]:
resnet.summary()

In [ ]:
!wget -q "https://s1.1zoom.ru/big0/607/359313-sepik.jpg" -O dog.jpg

In [ ]:
dog = image.load_img('./dog.jpg', target_size=(224, 224))
plt.imshow(dog)

img = image.img_to_array(dog)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

preds = resnet.predict(img)
print('Predicted:', decode_predictions(preds, top=3)[0])


In [ ]:
resnet.layers[-1].trainable

## тексты

In [ ]:
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import SimpleRNN

In [ ]:
num_words = 1000
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=num_words)

In [ ]:
max_review_length = 100
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:
model = Sequential()
model.add(Embedding(num_words, 16, input_length=max_review_length))
model.add(SimpleRNN(10))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

## Домашнее задание
 * Доучите любую модель из keras.applications на бинарную классификацию на данных https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition и сделайте посылку в рейтинговую систему. В качестве отчета присылайте свой код и посылку на kaggle. 
 
 Подсказка - доучивать на датасете всю сеть целиком долго, плюс она переобучится.
 * Реализуйте сверточную нейронную сеть на данных IMDB. Сравните по качеству и времени работы с рекуррентной сетью выше при примерно равном числе параметров. В качестве отчета присылайте свой код с обучением моделей и обсуждение итогов.